In [1]:
import sqlite3
import os
import sys
import openai
import json

from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.schema import AIMessage, HumanMessage
from tqdm import tqdm
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
sys.path.insert(0, ".")
sys.path.insert(1, "..")
from src.config import AZURE_OPENAI_KEY, AZURE_OPENAI_ENDPOINT, AZURE_ENGINE_NAME

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import re

In [2]:
openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

In [3]:
model = ChatOpenAI(engine=AZURE_ENGINE_NAME, openai_api_key=AZURE_OPENAI_KEY, openai_api_base=AZURE_OPENAI_ENDPOINT, temperature=0.5)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [4]:
conn = sqlite3.connect('../db/sword_hackathon.db')

# Create a cursor
cursor = conn.cursor()

# Create a new table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS youtube_summaries (
        id INTEGER PRIMARY KEY,
        url TEXT,
        summary TEXT
    )
""")

# Commit the changes
conn.commit()



In [5]:
# Connect to the SQLite database
conn = sqlite3.connect('../db/sword_hackathon.db')

# Execute the SQL command and convert it to a pandas DataFrame
df_existing = pd.read_sql_query("SELECT * FROM youtube_summaries", conn)

# Close the connection to the database
conn.close()

# Display the DataFrame
df_existing.iloc[0]['summary']

'"{\\n    \\"Topic\\": \\"Butter Chicken Recipe\\",\\n    \\"Summary\\": \\"Learn how to make delicious butter chicken at home with this easy recipe. Marinate chicken with garlic paste, ginger paste, red chilli powder, and salt. Pan fry the chicken until almost done. In a separate pan, saut\\u00e9 roughly cut onions and add butter. Add roughly cut tomatoes and cashew nuts. Mix well and add water, garlic paste, salt, malt vinegar, sugar, garam masala powder, and red chilli powder. Simmer the mixture until mushy and blend it into a fine puree. Strain the mixture and reheat it with butter, cream, and crushed methi. Let the gravy simmer and serve with fresh cream and kasoori methi.\\",\\n    \\"Wikipedia\\": \\"Butter Chicken\\",\\n    \\"Google\\": \\"Butter Chicken recipe\\",\\n    \\"Arxiv\\": \\"NA\\"\\n}"'

In [6]:
def process_summary(text):
    start_message = "I'm providing you with a YouTube transcription and the title: {transcription}."
    prompt_start = """Please generate an output in JSON format. If a requested field doesn't make sense, please fill it with 'NA'. The maximum word count for each field is indicated after each field placeholder: 
                    {{
                        "Topic": "A topic derived from the video transcription, intended for use in a recommendation system (up to 10 words)",
                        "Summary": "A summary of the video transcription (up to 100 words)",
                        "Wikipedia": "Keyword to be used for deeper research on Wikipedia relating to the video (up to 5 words)",
                        "Google": "Keyword to be used for further searches on Google about the video (up to 5 words)",
                        "Arxiv": "Keyword to search for academic papers related to the video's topic (up to 5 words)"
                    }}
    
                    Transcription:
                    {text}
                """
    final_prompt = prompt_start.format(text=text)
    
    prompt = [AIMessage(content=start_message), HumanMessage(content=final_prompt)]
    model
    response = model(prompt).content

    try:
        json.loads(response)
    except ValueError as e:
        response = None
        msg = 'Output not JSON.'

    return response

In [7]:
conn = sqlite3.connect('../db/sword_hackathon.db')

# Create a cursor
cursor = conn.cursor()

# Fetch all rows
cursor.execute("SELECT * FROM youtube_content ORDER BY id")
rows = cursor.fetchall()

# Iterate over all rows
for row in tqdm(rows):
    if row[-1] not in df_existing.url.unique():
        try:
            summary = process_summary(row[3])
        except:
            continue
        if summary is not None:
            json_summary = json.dumps(summary)
            cursor.execute("INSERT INTO youtube_summaries (url, summary) VALUES (?, ?)", (row[-1], json_summary))

# Commit the changes
conn.commit()
conn.close()


100%|█████████████████████████████████████████| 424/424 [01:43<00:00,  4.10it/s]
